In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
import numba

# Origin in bottom left
LEFT = np.array([0, 0])
RIGHT = np.array([0, 1])
TOP = np.array([np.sqrt(3)/2, 0.5])
CORNERS = [LEFT, RIGHT, TOP]

In [ ]:
def generate_triangle(seed: np.ndarray, iters: int) -> np.ndarray:
    """shape (2, iters)"""
    out = np.empty((2, iters))
    out[:, 0] = seed

    for i in range(1, iters):
        out[:, i] = (random.choice(CORNERS) + out[:, i-1])/2
    
    return out

In [ ]:
t = generate_triangle(np.array([0.1, 0.1]), 2000)

plt.figure()
plt.scatter(t[1, :], t[0, :])
plt.axis("off")
plt.show()

In [ ]:
@numba.jit(nopython=True, parallel=True)
def rasterize(triangle: np.ndarray, shape: tuple[int, int]) -> np.ndarray:
    out = np.zeros(shape, dtype=np.byte)
    Ny, Nx = shape
    for i in numba.prange(triangle.shape[1]):
        py, px = triangle[:, i]
        x = int(np.floor(px * Nx))
        y = int(np.floor(py * Ny))
        out[y, x] = 1
    return out

In [ ]:
t = generate_triangle(np.array([0.1, 0.1]), 200000)
tr = rasterize(t, (400, 400))

plt.figure()
plt.subplot(1,2,1)
plt.scatter(t[1, :], t[0, :])
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(tr, origin="lower")
plt.axis("off")
plt.show()

In [ ]:
def fractal_dimension(t: np.ndarray, n_boxes: int) -> float:
    return np.count_nonzero(rasterize(t, (n_boxes, n_boxes))) / n_boxes**2

In [ ]:
t = generate_triangle(np.array([0.1, 0.1]), 2000000)
d = []
boxes = np.logspace(1, 12, 30, endpoint=True, base=2, dtype=np.int64)

for boxcount in boxes:
    d.append(fractal_dimension(t, boxcount))

slope = np.polyfit(np.log2(boxes), np.log2(d), deg=1)
slope = slope[0]

plt.figure()
plt.loglog(boxes, d)
plt.xlabel("Boxcount")
plt.ylabel("Box population fraction")
plt.legend([f"{slope = :.2f} => dim = {2 + slope :.2f}"])
plt.show()